In [2]:
import os
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import pandas as pd
import shutil
import time

In [3]:
# define folder paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")

# open rgi df
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [13]:
# get all the data for the 5-year average products (infilled and not infilled)
folder_products = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Average AAs')

# get list of all the .tif AAs in the folder
all_tifs_names = os.listdir(folder_products)

# keep only unfilled tif file paths, make them into full paths
all_tifs_names = [i for i in all_tifs_names if i[-9:]=='final.tif']
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]
all_rgi_id = [i[3:17] for i in all_tifs_names] # get rgiid
df = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths, 'year':12345})

# keep only infilled tif file paths, make them into full paths
all_tifs_names = os.listdir(folder_products)
all_tifs_names = [i for i in all_tifs_names if i[-10:]=='infill.tif']
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]
all_rgi_id = [i[3:17] for i in all_tifs_names] # get rgiid
df_infill = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths, 'year':12345})

### add in the products for each individual year
folder_products = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Annual AAs')
all_tifs_names = os.listdir(folder_products)
all_tifs_names = [i for i in all_tifs_names if i[-9:]=='final.tif']
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]
all_rgi_id = [i[3:17] for i in all_tifs_names]
all_years = [int(i[18:22]) for i in all_tifs_names]
df2 = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths, 'year':all_years})

### add in the products for each individual year, infilled
folder_products = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Annual AAs')
all_tifs_names = os.listdir(folder_products)
all_tifs_names = [i for i in all_tifs_names if i[-10:]=='infill.tif']
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]
all_rgi_id = [i[3:17] for i in all_tifs_names]
all_years = [int(i[18:22]) for i in all_tifs_names]
df2_infill = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths, 'year':all_years})

# concat
df_all = pd.concat([df, df_infill, df2, df2_infill])
df_infill_all = pd.concat([df_infill, df2_infill])
df_nofill_all = pd.concat([df, df2])

# add o2 region as column
df_all = df_all.merge(rgi_gdf, on='RGIId', how='left')
df_infill_all = df_infill_all.merge(rgi_gdf, on='RGIId', how='left')
df_nofill_all = df_nofill_all.merge(rgi_gdf, on='RGIId', how='left')
print(len(df))
print(len(df_infill))
print(len(df2))
print(len(df2_infill))
print(len(df_all))
df_all
# df_all['O2Region'].unique()

3002
3002
14900
14827
35731


,RGIId,tif_name,tif_path,year,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,...,Status,Connect,Form,TermType,Surging,Linkages,Name,O3Region,O3_name,geometry
0,RGI60-01.00004,S2_RGI60-01.00004_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G213880E63381N,20090703,-9999999,-146.120,63.3810,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((390852.892 1513270.533, 390859.011 1..."
1,RGI60-01.00005,S2_RGI60-01.00005_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G212943E63551N,20090703,-9999999,-147.057,63.5510,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((344516.095 1527311.059, 344445.809 1..."
2,RGI60-01.00006,S2_RGI60-01.00006_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G213756E63571N,20090703,-9999999,-146.244,63.5710,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((383858.051 1532357.663, 383852.553 1..."
3,RGI60-01.00010,S2_RGI60-01.00010_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G212830E63513N,20090703,-9999999,-147.170,63.5130,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((339736.019 1522275.072, 339760.103 1..."
4,RGI60-01.00012,S2_RGI60-01.00012_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G213349E63366N,20090703,-9999999,-146.651,63.3660,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((367276.652 1509860.919, 367276.151 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35726,RGI60-01.27110,S2_RGI60-01.27110_2018_AA_final_infill.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2018,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."
35727,RGI60-01.27110,S2_RGI60-01.27110_2019_AA_final_infill.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2019,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."
35728,RGI60-01.27110,S2_RGI60-01.27110_2020_AA_final_infill.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2020,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."
35729,RGI60-01.27110,S2_RGI60-01.27110_2021_AA_final_infill.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2021,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."


In [12]:
# check for duplicates
all_names = list(df2['RGIId'].values)
infill_names = list(df2_infill['RGIId'].values)

list(set(all_names).difference(infill_names))

['RGI60-01.22197',
 'RGI60-01.08415',
 'RGI60-01.00592',
 'RGI60-01.17196',
 'RGI60-01.25542',
 'RGI60-01.17359',
 'RGI60-01.16327',
 'RGI60-01.07055',
 'RGI60-01.22152',
 'RGI60-01.01223']

In [20]:
# choose if you want to use infill or unfilled, and corresponding labels
use_infill = 0
if use_infill:
    df_to_use = df_infill_all
    labels = ["all_infill", "2018_infill","2019_infill","2020_infill","2021_infill","2022_infill"]
    ys = [12345, 2018, 2019, 2020, 2021, 2022]
else:
    df_to_use = df_nofill_all
    labels = ["all", "2018","2019","2020","2021","2022"]
    ys = [12345, 2018, 2019, 2020, 2021, 2022]


##### this is what we will actually use.
# all the infilled products, and then the 5-year average unfilled
new_df = df.copy().merge(rgi_gdf, on='RGIId', how='left')
new_df['year'] = 99999
df_to_use = pd.concat([df_infill_all, new_df])
labels = ["all_nofill", "all", "2018", "2019", "2020", "2021", "2022"]
ys = [99999, 12345, 2018, 2019, 2020, 2021, 2022]
use_infill = 0

for i in range(len(ys)):
    for n in [2,3,4,5,6]:
        
        label = labels[i]
        y = ys[i]
        print(label,n)

        # keep only the rgi ids that are in o2region x and the given years
        df_subset = df_to_use[ (df_to_use['O2Region']==str(n)) & (df_to_use['year']==y) ]
        print(len(df_subset))

        # grab the full list of paths
        all_paths = df_subset['tif_path'].values

        # define path for vrt, output location/name
        if use_infill:
            out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"O2Region_0{n}_{label}_AA_infill.tif")
            vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f'merged{n}_{label}_infill.vrt')
        else:
            out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"O2Region_0{n}_{label}_AA.tif")
            vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f'merged{n}_{label}.vrt')

        # build vrt dataset
        vrt = gdal.BuildVRT(vrt_path, list(all_paths), VRTNodata='nan', srcNodata=0)    
        vrt = 0

print("Done")

all_nofill 2
656
all_nofill 3
176
all_nofill 4
483
all_nofill 5
541
all_nofill 6
1146
all 2
656
all 3
176
all 4
483
all 5
541
all 6
1146
2018 2
649
2018 3
174
2018 4
483
2018 5
536
2018 6
1136
2019 2
651
2019 3
174
2019 4
483
2019 5
535
2019 6
1135
2020 2
643
2020 3
173
2020 4
483
2020 5
531
2020 6
1108
2021 2
647
2021 3
171
2021 4
481
2021 5
536
2021 6
1135
2022 2
647
2022 3
171
2022 4
483
2022 5
535
2022 6
1127
Done


In [21]:
for i in range(len(ys)):
    for n in [2,3,4,5,6]:
        
        label = labels[i]
        y = ys[i]
        print(label,n)
        
        # define path for vrt, output location/name
        out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"O2Region_0{n}_{label}_AA.tif")
        vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f'merged{n}_{label}.vrt')

        # Open the VRT dataset
        ds = gdal.Open(vrt_path)

#         print(f"Mosaicing Region {n}")
        gdal.Translate(out_path, ds, outputType=gdal.GDT_Byte, creationOptions = ['PREDICTOR=2','COMPRESS=DEFLATE', 'ZLEVEL=9'])
        print("mosaicing done")
        
print("All done!")

all_nofill 2
Mosaicing Region 2
mosaicing done
all_nofill 3
Mosaicing Region 3
mosaicing done
all_nofill 4
Mosaicing Region 4
mosaicing done
all_nofill 5
Mosaicing Region 5
mosaicing done
all_nofill 6
Mosaicing Region 6
mosaicing done
all 2
Mosaicing Region 2
mosaicing done
all 3
Mosaicing Region 3
mosaicing done
all 4
Mosaicing Region 4
mosaicing done
all 5
Mosaicing Region 5
mosaicing done
all 6
Mosaicing Region 6
mosaicing done
2018 2
Mosaicing Region 2
mosaicing done
2018 3
Mosaicing Region 3
mosaicing done
2018 4
Mosaicing Region 4
mosaicing done
2018 5
Mosaicing Region 5
mosaicing done
2018 6
Mosaicing Region 6
mosaicing done
2019 2
Mosaicing Region 2
mosaicing done
2019 3
Mosaicing Region 3
mosaicing done
2019 4
Mosaicing Region 4
mosaicing done
2019 5
Mosaicing Region 5
mosaicing done
2019 6
Mosaicing Region 6
mosaicing done
2020 2
Mosaicing Region 2
mosaicing done
2020 3
Mosaicing Region 3
mosaicing done
2020 4
Mosaicing Region 4
mosaicing done
2020 5
Mosaicing Region 5
mosaic

In [22]:
### now make stitch together the subregions for each year into region-wide mosaics
# first make vrts
for i in range(len(ys)):
    
    label = labels[i]
    y = ys[i]
    print(label, y)
    
    # get list of paths to the 5 subregion mosaics
    all_paths = [os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"O2Region_0{n}_{label}_AA.tif") for n in [2,3,4,5,6] ]

    out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"allregions_{label}_AA.tif")
    vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f'allregions_{label}.vrt')
   
    # build vrt dataset
    vrt = gdal.BuildVRT(vrt_path, list(all_paths), VRTNodata='nan', srcNodata=0)    
    vrt = 0

print("VRT creation done")

# then use gdal to make the mosaic tifs
for i in range(len(ys)):
    
    label = labels[i]
    y = ys[i]
    
    out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"allregions_{label}_AA.tif")
    vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f'allregions_{label}.vrt')

    # Open the VRT dataset
    ds = gdal.Open(vrt_path)

    print(f"Mosaicing Year {y}")
    gdal.Translate(out_path, ds, outputType=gdal.GDT_Byte, creationOptions = ['PREDICTOR=2','COMPRESS=DEFLATE', 'ZLEVEL=9'])
    print("mosaicing done")
        
print("All done!")

VRT creation done
Mosaicing Year 99999
mosaicing done
Mosaicing Year 12345
mosaicing done
Mosaicing Year 2018
mosaicing done
Mosaicing Year 2019
mosaicing done
Mosaicing Year 2020
mosaicing done
Mosaicing Year 2021
mosaicing done
Mosaicing Year 2022
mosaicing done
All done!
